# "Introducing fastlinkcheck"

> Say goodbye broken links on your static sites.  Platform independent, fast, and built in python.
- author: "<a href='https://twitter.com/jeremyphoward'>Jeremy Howard</a> and <a href='https://twitter.com/HamelHusain'>Hamel Husain</a>"
- toc: false
- image: images/copied_from_nb/fastlinkcheck_images/fastlinkcheck.png
- comments: true
- categories: [nbdev, fastlinkcheck]
- permalink: /fastlinkcheck/
- badges: true

![diagram](fastlinkcheck_images/fastlinkcheck.png)

# Motivation

Recently, fastai has been hard at work improving and overhauling [nbdev](https://nbdev.fast.ai/), a literate programming environment for python.  A key feature of nbdev is automated generation of documentation from Jupyter notebooks.  This documentation system adds many niceties, such as the following types of hyperlinks automatically:

 - Links to source code on GitHub.
 - Links to both internal and external documentation by introspecting variable names in backticks.

Because documentation is so easy to create and maintain in nbdev, we find ourselves and others creating much more of it!  In addition to automatic hyperlinks, we often include our own links to relevant websites, blogs and videos when documenting code.  For example, one of the largest nbdev generated sites, [docs.fast.ai](https://docs.fast.ai/), has more than 300 external and internal links at the time of this writing.  


# The Solution

Due to the continued popularity of [fastai](https://github.com/fastai/fastai) and the growth of new nbdev projects, grooming these links manually became quite tedious.  We investigated solutions that could verify links for us automatically, but were not satisfied with any existing solutions.  These are the features we desired:

- A platform independent solution that is not tied to a specific static site generator like Jekyll or Hugo.
- Intelligent introspection of external links that are actually internal links.  For example, if we are building the site docs.fast.ai, a link to `https://docs.fast.ai/tutorial` should not result in a web request, but rather introspection of the local file system for the presence of `tutorial.html` in the right location.
- Verification of any links to assets like CSS, data, javascript or other files. 
- Logs that are well organized that allow us to see each broken link or reference to a non-existent path, and the pages these are found in.
- Parallelism to verify links as fast as possible. 
- Lightweight, easy to install with minimal dependencies.

We tried tools such as [linkchecker](https://github.com/wummel/linkchecker) and [pylinkvalidator](https://github.com/bartdag/pylinkvalidator), but these required your site to be first be hosted. Since we wanted to check links on a static site, hosting is overhead we wanted to avoid.

This is what led us to create [fastlinkcheck](https://fastlinkcheck.fast.ai/), which we discuss below.

> Note: For Ruby users, [htmlproofer](https://github.com/gjtorikian/html-proofer) apperas to provide overlapping functionality.  We have not tried this library. 

# A tour of fastlinkcheck

For this tour we will be referring to the files in the [fastlinkcheck repo](https://github.com/fastai/fastlinkcheck).  You should clone this repo in the current directory in order to follow along:

In [1]:
git clone https://github.com/fastai/fastlinkcheck.git
cd fastlinkcheck

SyntaxError: invalid syntax (<ipython-input-1-fdb0584cc2b5>, line 1)

## Installation

You can install `fastlinkcheck` with pip:

`pip install fastlinkcheck`

## Usage

After installing `fastlinkcheck`, the cli command `link_check` is available from the command line.  We can see various options with the `--help` flag.

In [ ]:
link_check --help

From the root of [fastlinkcheck repo](https://github.com/fastai/fastlinkcheck), We can search the directory `_example/broken_links` recursively for broken links like this:

In [ ]:
link_check _example/broken_links 

Specifying the `--host` parameter allows you detect links that are internal by identifying links with that host name. External links are verified by making a request to the appropriate website.  On the other hand, internal links are verified by inspecting the presence and content of local files. 

We must be careful when using the `--host` argument to only pass the host (and path, if necessary) **without** the protocol.  For example, this is how we specify the hostname if your site's url is `http://fastlinkcheck.com/test.html`:

In [ ]:
link_check _example/broken_links --host fastlinkcheck.com

We now have one less broken link as there is indeed a file named `test.html` in the root of the path we are searching.  However, if we add a path to the end of `--host` , such as `fastlinkcheck.com/mysite` the link would again be listed as broken because `_example/broken_links/mysite/test.html` does not exist:

In [2]:
link_check _example/broken_links --host fastlinkcheck.com/mysite

SyntaxError: invalid syntax (<ipython-input-2-c2a9e65a02aa>, line 1)

You can ignore links by creating a text file that contains a list of urls and paths to ignore.  For example, the file `_example/broken_links/linkcheck.rc` contains:

In [ ]:
cat _example/broken_links/linkcheck.rc

We can use this file to ignore urls and paths with the `--config_file` argument.  This will filter out references to the broken link `/test.js` from our earlier results:

In [3]:
link_check _example/broken_links --host fastlinkcheck.com --config_file _example/broken_links/linkcheck.rc

SyntaxError: invalid syntax (<ipython-input-3-f6688a56e6b4>, line 1)

Finally, if there are no broken links, `link_check` will not return anything.  The directory `_example/no_broken_links/` does not contain any HTML files with broken links:

In [ ]:
link_check _example/no_broken_links

## Python

You can also use these utilities from python instead of the terminal.  Please see [these docs](https://fastlinkcheck.fast.ai/linkcheck.html/) for more information.

## Using `link_check` in GitHub Actions


The `link_check` CLI utility that is installed with `fastlinkcheck` can be very useful in continuous integration systems like [GitHub Actions](https://github.com/features/actions).  Here is an example GitHub Actions workflow that uses `link_check`:


```yaml
name: Check Links
on: [workflow_dispatch, push]

jobs:
  check-links:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v2
    - uses: actions/setup-python@v2
    - name: check for broken links
      run: |
        pip install fastlinkcheck
        link_check _example 
```

We can a few more lines of code to open an issue instead when a broken link is found, using the [gh cli](https://github.com/cli/cli):

```yaml
...
    - name: check for broken links
      run: |
        pip install fastlinkcheck
        link_check _example 2> err || true
        export GITHUB_TOKEN="YOUR_TOKEN"
        [[ -s err ]] &&  gh issue create -t "Broken links found" -b "$(< err)" -R "yourusername/yourrepo"
```

We can extend this even further to only open an issue when another issue with a specific label isn't already open:

```yaml
...
    - name: check for broken links
      run: |
       pip install fastlinkcheck
       link_check "docs/_site" --host "docs.fast.ai" 2> err || true
        export GITHUB_TOKEN="YOUR_TOKEN"
        if [[ -z $(gh issue list -l "broken-link")) && (-s err) ]]; then
          gh issue create -t "Broken links found" -b "$(< err)" -l "broken-link" -R "yourusername/yourrepo"
        fi
```


See the [GitHub Actions docs](https://docs.github.com/en/free-pro-team@latest/actions) for more information.

# Resources

The following resources are relevant for those interested in learning more about `fastlinkcheck`:

- The fastlinkcheck [GitHub repo](https://github.com/fastai/fastlinkcheck)
- The fastlinkcheck [docs](https://fastlinkcheck.fast.ai/)